In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
np.random.seed(1)

In [3]:
ball = pd.read_csv('Half Ball Prediction Data.csv')
convex = pd.read_csv('Convex Prediction Data.csv')
freeform = pd.read_csv('Freeform 2 Prediction Data.csv')

test_data = pd.read_csv("Whole Test Grid.csv")

test_total = []
for i in range(1,51):
    nameX = "Scan" + str(i) + "X"
    nameY = "Scan" + str(i) + "Y"
    nameZ = "Scan" + str(i) + "Z"
    nameCurv = "Scan" + str(i) + "curvature"
    nameAngle = "Scan" + str(i) + "angle"
    scan_i = test_data[[nameX, nameY, nameZ,nameCurv,nameAngle]]
    scan_i.columns = ['X', 'Y', 'Z','curvature','angle']
    test_total.append(scan_i)

test_d = []
for i in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35]:
    test_d.append(test_total[i])
    
true_variance = pd.read_csv('True Variance.csv')

In [4]:
def hyperbolic(x):
    return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))

def generate_neurons_uniform(X,num_neurons):
    k = X.shape[1]
    variances = np.std(X,axis=0)
    weights = []
    for i in range(k):
        λ = variances[i]/np.sum(variances)
        a = 2.5 * λ / np.max(np.abs(X[:,i]))
        weight = np.random.uniform(low=-a,high=a,size=num_neurons)
        weights.append(weight)
    weights = np.array(weights)
    intercept = np.random.uniform(low=-1,high=1,size=num_neurons)
    
    return weights, intercept

def cal_neuron_val(X, weights, intercept):
    neuron_val = np.matmul(X,weights) + intercept
    neuron_val = hyperbolic(neuron_val)
    return neuron_val


In [5]:
def mbelm(data,coordinate,num_neurons=50):
    weights, intercept = generate_neurons_uniform(data[['Mean Curvature','Mean Angle']].values,num_neurons=num_neurons)
    neuron_val = cal_neuron_val(data[['Mean Curvature','Mean Angle']].values,weights,intercept)
    clf = BayesianRidge()
    name = 'Pointwise Variance ' + coordinate
    clf.fit(neuron_val, data[name].values)
    return {'weight':weights,'intercept':intercept,'model':clf}

In [6]:
def mbelm_pred(train,model):
    neuron_pred = cal_neuron_val(train[['curvature','angle']].values, model['weight'],model['intercept'])
    return model['model'].predict(neuron_pred)

In [7]:
ball['Pointwise Variance X'] = np.log(ball['Pointwise Variance X'])
ball['Pointwise Variance Y'] = np.log(ball['Pointwise Variance Y'])
ball['Pointwise Variance Z'] = np.log(ball['Pointwise Variance Z'])

In [8]:
convex['Pointwise Variance X'] = np.log(convex['Pointwise Variance X'])
convex['Pointwise Variance Y'] = np.log(convex['Pointwise Variance Y'])
convex['Pointwise Variance Z'] = np.log(convex['Pointwise Variance Z'])

In [9]:
freeform['Pointwise Variance X'] = np.log(freeform['Pointwise Variance X'])
freeform['Pointwise Variance Y'] = np.log(freeform['Pointwise Variance Y'])
freeform['Pointwise Variance Z'] = np.log(freeform['Pointwise Variance Z'])

In [10]:
ball_x = mbelm(ball,'X')
ball_y = mbelm(ball,'Y')
ball_z = mbelm(ball,'Z')

convex_x = mbelm(convex,'X')
convex_y = mbelm(convex,'Y')
convex_z = mbelm(convex,'Z')

freeform_x = mbelm(freeform,'X')
freeform_y = mbelm(freeform,'Y')
freeform_z = mbelm(freeform,'Z')

In [11]:
X_rmse = pd.DataFrame()
Y_rmse = pd.DataFrame()
Z_rmse = pd.DataFrame()

for i in range(30):
    ball_x_rmse = sqrt(mean_squared_error(true_variance['X Variance'], np.exp(mbelm_pred(test_d[i],ball_x))))
    ball_x_rmse = pd.DataFrame({'RMSE':[ball_x_rmse],'Training history':'Ball'})
    
    convex_x_rmse = sqrt(mean_squared_error(true_variance['X Variance'], np.exp(mbelm_pred(test_d[i],convex_x))))
    convex_x_rmse = pd.DataFrame({'RMSE':[convex_x_rmse],'Training history':'Freeform 1'})
    
    freeform_x_rmse = sqrt(mean_squared_error(true_variance['X Variance'], np.exp(mbelm_pred(test_d[i],freeform_x))))
    freeform_x_rmse = pd.DataFrame({'RMSE':[freeform_x_rmse],'Training history':'Freeform 2'})

    
    X_rmse = X_rmse.append(ball_x_rmse)
    X_rmse = X_rmse.append(convex_x_rmse)
    X_rmse = X_rmse.append(freeform_x_rmse)
    
    ball_y_rmse = sqrt(mean_squared_error(true_variance['Y Variance'], np.exp(mbelm_pred(test_d[i],ball_y))))
    ball_y_rmse = pd.DataFrame({'RMSE':[ball_y_rmse],'Training history':'Ball'})
    
    convex_y_rmse = sqrt(mean_squared_error(true_variance['Y Variance'], np.exp(mbelm_pred(test_d[i],convex_y))))
    convex_y_rmse = pd.DataFrame({'RMSE':[convex_y_rmse],'Training history':'Freeform 1'})
    
    freeform_y_rmse = sqrt(mean_squared_error(true_variance['Y Variance'], np.exp(mbelm_pred(test_d[i],freeform_y))))
    freeform_y_rmse = pd.DataFrame({'RMSE':[freeform_y_rmse],'Training history':'Freeform 2'})
    

    Y_rmse = Y_rmse.append(ball_y_rmse)
    Y_rmse = Y_rmse.append(convex_y_rmse)
    Y_rmse = Y_rmse.append(freeform_y_rmse)

    
    ball_z_rmse = sqrt(mean_squared_error(true_variance['Y Variance'], np.exp(mbelm_pred(test_d[i],ball_z))))
    ball_z_rmse = pd.DataFrame({'RMSE':[ball_z_rmse],'Training history':'Ball'})
    
    convex_z_rmse = sqrt(mean_squared_error(true_variance['Y Variance'], np.exp(mbelm_pred(test_d[i],convex_z))))
    convex_z_rmse = pd.DataFrame({'RMSE':[convex_z_rmse],'Training history':'Freeform 1'})
    
    freeform_z_rmse = sqrt(mean_squared_error(true_variance['Y Variance'], np.exp(mbelm_pred(test_d[i],freeform_z))))
    freeform_z_rmse = pd.DataFrame({'RMSE':[freeform_z_rmse],'Training history':'Freeform 2'})
 
    
    Z_rmse = Z_rmse.append(ball_z_rmse)
    Z_rmse = Z_rmse.append(convex_z_rmse)
    Z_rmse = Z_rmse.append(freeform_z_rmse)


In [12]:
# X-coordiante RMSE means and standard deviations
print('X-coordinates:')
print(np.mean(X_rmse.loc[X_rmse['Training history']=='Ball','RMSE']),'|',
      np.mean(X_rmse.loc[X_rmse['Training history']=='Freeform 1','RMSE']),'|',
      np.mean(X_rmse.loc[X_rmse['Training history']=='Freeform 2','RMSE']))
print()
print(np.std(X_rmse.loc[X_rmse['Training history']=='Ball','RMSE']),'|',
      np.std(X_rmse.loc[X_rmse['Training history']=='Freeform 1','RMSE']),'|',
      np.std(X_rmse.loc[X_rmse['Training history']=='Freeform 2','RMSE']))

print()
print()


# Y-coordinate RMSE means and standard deviations
print('Y-coordinates:')
print(np.mean(Y_rmse.loc[Y_rmse['Training history']=='Ball','RMSE']),'|',
      np.mean(Y_rmse.loc[Y_rmse['Training history']=='Freeform 1','RMSE']),'|',
      np.mean(Y_rmse.loc[Y_rmse['Training history']=='Freeform 2','RMSE']))
print()
print(np.std(Y_rmse.loc[Y_rmse['Training history']=='Ball','RMSE']),'|',
      np.std(Y_rmse.loc[Y_rmse['Training history']=='Freeform 1','RMSE']),'|',
      np.std(Y_rmse.loc[Y_rmse['Training history']=='Freeform 2','RMSE']))

print()
print()

# Z-coordinate RMSE means and standard deviations
print('Z-coordinates:')
print(np.mean(Z_rmse.loc[Z_rmse['Training history']=='Ball','RMSE']),'|',
      np.mean(Z_rmse.loc[Z_rmse['Training history']=='Freeform 1','RMSE']),'|',
      np.mean(Z_rmse.loc[Z_rmse['Training history']=='Freeform 2','RMSE']))
print()
print(np.std(Z_rmse.loc[Z_rmse['Training history']=='Ball','RMSE']),'|',
      np.std(Z_rmse.loc[Z_rmse['Training history']=='Freeform 1','RMSE']),'|',
      np.std(Z_rmse.loc[Z_rmse['Training history']=='Freeform 2','RMSE']))

X-coordinates:
0.011954141530370811 | 0.001746997477962877 | 0.0016733444218047589

0.0033160817244425383 | 0.00023528084948939754 | 2.6401427068064373e-06


Y-coordinates:
0.030924908673582566 | 0.0034447441679616927 | 0.0032830673631694773

0.001601695762644734 | 8.762279368823588e-06 | 8.553874323779124e-05


Z-coordinates:
0.5241614208824116 | 0.004805990027338509 | 0.006346560802769513

0.08572545121377577 | 0.00024182102259214067 | 0.00041063842561660707
